In [ ]:
import os

import pandas as pd
import geopandas as gpd
import numpy as np

import psycopg2
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import GeoJSONDataSource
from bokeh.models.tools import LassoSelectTool, BoxSelectTool
import xyzservices.providers as xyz
from bokeh.palettes import Bokeh8

from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.layouts import column, row


In [ ]:

import xyzservices.providers as xyz
# 'USImagery', 'USImageryTopo', 'USTopo'
tiles = xyz['USGS']['USTopo']

In [ ]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'validation/data/')
DATA_DIR

In [ ]:
def get_catchment_polygons(region):
    file_dir = os.path.join(BASE_DIR, f'processed_data/derived_basins/{region}/')
    fpath = os.path.join(file_dir, f'{region}_basins_R0.parquet')
    df = gpd.read_parquet(fpath)
    df.set_geometry('basin_geometry', inplace=True)
    return df
    

def get_boundary_deviations(region):
    deviations_path = f'{region}_bounds_deviations_test.geojson'
    deviations = gpd.read_file(os.path.join(DATA_DIR, deviations_path))
    deviations = deviations[['geometry']].dissolve().explode(index_parts=False)
    return deviations

In [ ]:
rc = 'FRA'
deviation_df = get_boundary_deviations(rc)
ws_df = get_catchment_polygons(rc)

In [ ]:
print(f'There are {len(ws_df)} catchment polygons in the dataset')
assert deviation_df.crs == ws_df.crs

In [ ]:
deviation_df = deviation_df[deviation_df.is_valid]
ws_df = ws_df[ws_df.is_valid]

In [ ]:
ws_df.reset_index(inplace=True, drop=True)
deviation_df.reset_index(inplace=True, drop=True)

In [ ]:
# get the catchments that touch any of the deviation geometries
# intersecting = ws_df.sjoin(deviation_df, how='inner', predicate='touches')

In [ ]:
cols = ['ID', 'drainage_area_km2', 'ppt_lon_m_3005', 'ppt_lat_m_3005',
       'ppt_acc', 'Perimeter_km', 'id', 'Elevation_m', 'Aspect_deg',
       'Slope_deg', 'region_code', 'geometry', 'basin_geometry',
       'centroid_geometry', 'index_right']
# intersecting.drop_duplicates(subset=['ID'], keep='first', inplace=True)
# intersecting.head()
# intersecting.columns

In [ ]:
# intersecting.to_file('08A_test.geojson')

In [ ]:
query = """
SELECT 
    id, 
    drainage_area_km2, 
    inside_pct_area_FLAG, 
    outside_pct_area_FLAG, 
    region_code, 
    pour_pt
FROM 
    basins_schema.basin_attributes
WHERE 
    geometry_flag = TRUE;
"""


In [ ]:
conn = psycopg2.connect(
    dbname="basins",
    user="postgres",
    password="pgpass",
    host="localhost",
    port="5432"
)
gdf = gpd.read_postgis(query, conn, geom_col='pour_pt')
conn.close()

In [ ]:
def plot_log_line(c, k=1, x_range=(1, 1.25e3), y_range=(5, 1e4)):
    x = np.linspace(x_range[0], x_range[1], 100)
    # y = y_range[1] * (x / x_range[0])**(-1) * c 
    y =  k * (x / x_range[0])**(c)
    return x, y

In [ ]:
gdf['total_pct_area_flag'] = gdf['inside_pct_area_flag'] + gdf['outside_pct_area_flag']
gdf['area_uncertainty'] = gdf['total_pct_area_flag']#gdf['drainage_area_km2'] * (gdf['total_pct_area_flag'] / 100)

# Spatial Plot with Map Tiles
# Convert to Web Mercator for tile source
gdf = gdf.to_crs(epsg=3857)
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

In [ ]:
# Enable output to notebook
output_notebook()

# Create a ColumnDataSource
source = ColumnDataSource(gdf[['total_pct_area_flag', 'drainage_area_km2', 'area_uncertainty', 'x', 'y']].copy())

# Create a figure
plot = figure(title=f'Drainage Area vs Uncertain Area Flag (n={len(gdf)})',
              x_axis_label=r'$$\text{Drainage Area } [km^2 ]$$',
              y_axis_label=r'$$\text{Uncertain Area } [\%] $$', y_axis_type='log', x_axis_type='log')
# Add Lasso Select Tool
plot.add_tools(LassoSelectTool(), BoxSelectTool())
# cs = [0.65, 0.45, 0.3]
cono = 0
# for c in cs:
c = 1
color = Bokeh8[cono]
# cono += 1
x0, y0 = plot_log_line(c, k=0.1)
plot.line(x0, y0, line_dash='dashed', line_width=2, line_color=color, legend_label=f'{c}')
# Add a scatter renderer with circle markers
# plot.circle(x='drainage_area_km2', y='area_uncertainty', source=source, size=4, color="navy", alpha=0.5)
plot.scatter(x='drainage_area_km2', y='area_uncertainty', source=source, size=4, 
             selection_color="orange", nonselection_alpha=0.1, selection_alpha=1,
             nonselection_color='gray', color="navy", alpha=0.5)


In [ ]:
spatial_plot = figure(title='Point Locations', x_axis_type="mercator", y_axis_type="mercator",
                      x_axis_label='Longitude', y_axis_label='Latitude')

# dir(xyz)
spatial_plot.add_tile(tiles)
spatial_plot.scatter(x='x', y='y', source=source, size=4, color="navy", alpha=0.5, selection_alpha=1,
                     selection_color="orange", nonselection_alpha=0.9, nonselection_color='grey')
spatial_plot.add_tools(LassoSelectTool(), BoxSelectTool())
spatial_plot.xaxis.axis_label = r'$$\text{Area} [\text{km}^2$$'
spatial_plot.yaxis.axis_label = r'$$P(x)$$'
spatial_plot.xaxis.axis_label_text_font_size = '13px'
spatial_plot.yaxis.axis_label_text_font_size = '13px'
spatial_plot.xgrid.visible = False
spatial_plot.ygrid.visible = False

In [ ]:
# Show the plot
layout = row(plot, spatial_plot)
show(layout)